# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Предобработка не требуется

Сгенерирую случайную матрицу и проверю на обратимость

In [6]:
invert_random = np.random.rand(4, 4)
invert_random

array([[0.57270471, 0.73075407, 0.16831653, 0.30114462],
       [0.41072727, 0.70145835, 0.61725739, 0.99736286],
       [0.1001321 , 0.70926997, 0.11623191, 0.00940931],
       [0.03590669, 0.5191743 , 0.94398576, 0.55465379]])

In [7]:
invert = np.linalg.inv(invert_random)
invert

array([[ 2.92801439, -1.1422366 , -2.25483315,  0.50244861],
       [-0.5677198 ,  0.35812446,  1.91010484, -0.36813425],
       [ 1.0603079 , -1.3687581 , -1.13302374,  1.90479804],
       [-1.46272387,  2.06826869,  0.286385  , -1.12686086]])

In [8]:
df.columns = df.columns.str.lower()

In [9]:
df.columns

Index(['пол', 'возраст', 'зарплата', 'члены семьи', 'страховые выплаты'], dtype='object')

привычней воспринимать названия колонок без заглавных)

In [10]:
df_f = df.drop('страховые выплаты', axis=1)
df_t = df['страховые выплаты']

print('Размер признаков:', df_f.shape[0], ';', 'Количество признаков:', df_f.shape[1])
print('Размер целевого признака:', df_t.shape[0])

Размер признаков: 5000 ; Количество признаков: 4
Размер целевого признака: 5000


In [11]:
model = LinearRegression()
model.fit(df_f, df_t)
r2 = model.score(df_f, df_t)

print('R2 модели до:', r2)

R2 модели до: 0.42494550286668


In [12]:
inv_df_f = np.dot(df_f, invert)

model_2 = LinearRegression()
model_2.fit(inv_df_f, df_t)
r2 = model_2.score(inv_df_f, df_t)

print('R2 модели после:', r2)

R2 модели после: 0.4249455028666944


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:**

$$
a1 = X(X^T X)^{-1} X^T y = X(X^T)^{-1}X^{-1}X^Ty = XX^{-1}y
$$

$$
a2 = XP((XP)^T(XP))^{-1}(XP)^Ty = XP(XP)^{-1}((XP)^T)^{-1}(XP)^Ty = XPP^{-1}X^{-1}((XP)^T)^{-1}(XP)^Ty = XEX^{-1}(P^TX^T)^{-1}P^TX^Ty = XX^{-1}(X^T)^{-1}(P^T)^{-1}P^TX^Ty = XX^{-1}(X^T)^{-1}EX^Ty = XX^{-1}(X^T)^{-1}X^Ty = XX^{-1}y
$$



$$
w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} E X^T y
$$
\
$$
w' = P^{-1}w
$$

## Алгоритм преобразования

**Алгоритм**

1. Генерируем случайную обратимую матрицу P размером nxn, где n=количество нецелевых признаков
2. Умножаем P на матрицу признаков

## Проверка алгоритма

In [13]:
test2 = np.random.rand(4, 4)
test2

array([[0.83365714, 0.18476467, 0.16888311, 0.20399486],
       [0.59727368, 0.41540835, 0.1783363 , 0.82915287],
       [0.98207756, 0.63343877, 0.72737828, 0.79728415],
       [0.50178555, 0.60154818, 0.74703706, 0.78366873]])

In [14]:
test = np.linalg.inv(test2)
test

array([[ 5.77215816e+00,  1.05385835e-01, -6.49960305e+00,
         4.99848828e+00],
       [-8.13067752e+01, -3.08932821e+00,  1.21649671e+02,
        -9.93298098e+01],
       [ 2.99909580e+01, -5.06648525e-01, -4.46402416e+01,
         3.81450018e+01],
       [ 3.01265054e+01,  2.78687132e+00, -4.66636164e+01,
         3.79596202e+01]])

In [15]:
final = np.dot(df_f, test)

model_2 = LinearRegression()
model_2.fit(final, df_t)
r2 = model_2.score(final, df_t)

print('R2 модели после:', r2)

R2 модели после: 0.4249455028665228


Алгоритм проверен, все работает, качество модели от преобразований признаков не меняется